In [1]:
import numpy as np

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10*np.random.rand(N))
yerr = 0.1+0.5*np.random.rand(N)
y = m_true*x+b_true
y += np.abs(f_true*y) * np.random.randn(N)
y += yerr * np.random.randn(N)

In [6]:
A = np.vstack((np.ones_like(x), x)).T
C = np.diag(yerr * yerr)
cov = np.linalg.inv(np.dot(A.T, np.linalg.solve(C, A)))
b_ls, m_ls = np.dot(cov, np.dot(A.T, np.linalg.solve(C, y)))
print(m_ls, b_ls)

-1.23803004504 5.48091114242


In [4]:
def lnlike(theta, x, y, yerr):
    m, b, lnf = theta
    model = m * x + b
    inv_sigma2 = 1.0/(yerr**2 + model**2*np.exp(2*lnf))
    return -0.5*(np.sum((y-model)**2*inv_sigma2 - np.log(inv_sigma2)))

In [7]:
import scipy.optimize as op
nll = lambda *args: -lnlike(*args)
result = op.minimize(nll, [m_true, b_true, np.log(f_true)], args=(x, y, yerr))
m_ml, b_ml, lnf_ml = result["x"]
print(m_ml, b_ml)

-1.04734445696 4.67949889197


In [8]:
def lnprior(theta):
    m, b, lnf = theta
    if -5.0 < m < 0.5 and 0.0 < b < 10.0 and -10.0 < lnf < 1.0:
        return 0.0
    return -np.inf

In [9]:
def lnprob(theta, x, y, yerr):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta, x, y, yerr)

In [18]:
ndim, nwalkers = 3, 100
pos = [result["x"] + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
pos

[array([-1.04744937,  4.67933504, -0.77038232]),
 array([-1.04729867,  4.67944719, -0.77025051]),
 array([-1.04730807,  4.67942252, -0.77025469]),
 array([-1.04718202,  4.67942223, -0.77030562]),
 array([-1.04728521,  4.67949726, -0.77028325]),
 array([-1.04750949,  4.67940675, -0.77023172]),
 array([-1.04732939,  4.67961325, -0.77022364]),
 array([-1.0474002 ,  4.6797328 , -0.77037249]),
 array([-1.04740748,  4.67950463, -0.77033253]),
 array([-1.04728855,  4.67959215, -0.77010535]),
 array([-1.04750438,  4.67959006, -0.77032724]),
 array([-1.04715621,  4.67955193, -0.77036788]),
 array([-1.04751909,  4.67958829, -0.77030282]),
 array([-1.0474907 ,  4.6795393 , -0.77045217]),
 array([-1.04734235,  4.67943639, -0.77013029]),
 array([-1.0473969 ,  4.67924499, -0.77030974]),
 array([-1.04736832,  4.67955483, -0.77018072]),
 array([-1.04742076,  4.67951887, -0.77025934]),
 array([-1.04707797,  4.67936631, -0.77042443]),
 array([-1.04740075,  4.6795253 , -0.77042167]),
 array([-1.04719529,

In [11]:
import emcee
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(x, y, yerr))

In [12]:
sampler.run_mcmc(pos, 500)

(array([[-1.0869774 ,  4.87575747, -0.71685496],
        [-1.22642317,  5.6530539 , -0.9204933 ],
        [-0.9679656 ,  4.42042571, -0.63373035],
        [-1.10726583,  4.72813868, -0.902626  ],
        [-0.9375614 ,  4.01525272, -0.61717611],
        [-0.97324166,  4.47826018, -0.72136778],
        [-0.94628663,  4.34474671, -0.53838251],
        [-1.03800143,  4.51441662, -0.76522193],
        [-1.09351482,  4.78315261, -0.67309929],
        [-0.97904825,  4.37578904, -0.66303008],
        [-1.0436247 ,  4.57636088, -0.62449453],
        [-1.11278389,  4.89341012, -0.98278515],
        [-1.07526521,  4.92926794, -1.03280311],
        [-1.02720161,  4.55888144, -0.68111048],
        [-1.01763359,  4.36545734, -0.72106774],
        [-1.04455631,  4.52272785, -0.77382543],
        [-1.14204699,  5.08642741, -0.79601923],
        [-1.07017222,  4.77177728, -0.8130192 ],
        [-1.07466769,  4.91741441, -0.87137155],
        [-1.0189327 ,  4.55095794, -0.64440657],
        [-1.05433726